In [ ]:
import os, sys
import pandas as pd
import numpy as np

from pathlib import Path

# define the current path (notebooks in lab_utils)
currpath = os.getcwd()
labutilspath = str(Path(currpath).parents[1])
sys.path.append(labutilspath)

# import the autoscan routines
from autoscan import autoscan

pp = autoscan.postprocess()

def shear(vel, rho = 1):
    return rho * np.power(vel[1], 2)

def youngs(vel, rho = 1):
    v2 = np.power(vel, 2)
    E = rho *  np.product([v2[:, 1], 3 * v2[:, 0] - 4 * v2[:, 1]], axis = 0)
    E = np.divide(E, (v2[:, 0] - v2[:, 1]))
    return E

def lame(vel, rho = 1):
    v2 = np.power(vel, 2)
    l  = rho * (v2[:, 0] - 2 * v2[:, 1])
    return l

def bulk(vel, rho = 1):
    v2 = np.power(vel, 2)
    k  = rho * (v2[:, 0] - (4.0/3.0) * v2[:, 1])
    return k

def nu(vel, rho = 1):
    v2 = np.power(vel, 2)
    nu = np.divide(v2[:, 0] - 2 * v2[:, 1], 2 * (v2[:, 0] - v2[:, 1]))
    return nu


In [ ]:
datapath = '/home/urlab/sandbox/data/characterization/autoscan/autoscan.h5'
savepath = '/home/urlab/Documents/'

In [ ]:
df = pd.read_hdf(datapath, key = 'data')

In [ ]:
rocks = {'ah_001':  {'subtag': 's0', 'rho': 2.97},
         'sh_003':  {'subtag': 's0', 'rho': 2.584},
         'sh_001':  {'subtag': 's0', 'rho': 2.584},
         'wsg_003': {'subtag': 's3', 'rho': 2.323}, 
         'wsg_004': {'subtag': 's1', 'rho': 2.323}, 
        }

features = ['perm', ['vp_0', 'vs_0'], 'e_star']
velfeatures = ['M', 'k', 'E', 'nu']

In [ ]:
# for k, v in rocks.items():
#     for feature in features:
#         if isinstance(feature, str): feature = [feature]
#         dt = df.query("tag == '%s' & subtag == '%s' & instance == 'before'" % (k, v['subtag'])).loc[:, ['x', 'y'] + feature].copy()
#         dt.dropna(inplace = True)
#         dt.reset_index(inplace = True, drop = True)
#         if dt.shape[0] > 0:
#             if 'vp_0' in feature:
#                 feature = ['vels']
#                 vels = dt.loc[:, ['vp_0', 'vs_0']].values 
#                 rho  = v['rho']
#                 dt['M'] = dt.loc[:, ['vs_0']].apply(lambda x: rho * np.power(x, 2))
#                 dt['E'] = youngs(vels, rho)
#                 dt['l'] = lame(vels, rho)
#                 dt['k'] = bulk(vels, rho)
#                 dt['nu'] = nu(vels, rho)
                
#                 for m in dt.columns[-5:]:
#                     savefile = os.path.join(savepath, '_'.join(['data', k, v['subtag']] + feature + [m]) + '.csv')
#                     dt.loc[:, ['x', 'y', m]].to_csv(savefile, index = False)
            
#             savefile = os.path.join(savepath, '_'.join(['data', k, v['subtag']] + feature) + '.csv')
#             dt.to_csv(savefile, index = False)

In [ ]:
df.index.names

In [ ]:
# df.iloc[:, 2:].groupby(level = 'code').agg(['median', 'mean', 'std', 'min', 'max'])

In [ ]:
ftir_lambdas = pp.probe_settings['ftir']['lambdas'].shape
data = df.iloc[:, 2:]


In [ ]:
vels = data.loc[:, ['vp_0', 'vs_0']].copy()

# drop nan's
vels.dropna(inplace = True)

# remove vals<0
idx = np.logical_and((vels >= 1e3).all(axis = 1), (vels <= 7e3).all(axis = 1))
print("estimated correct values %.2f pct" % (100 * np.sum(idx) / vels.shape[0]))
print("samples to re-process: %s" % (vels.loc[idx == False, :].index.droplevel('m').unique()) )
vels = vels.loc[idx, :]

In [ ]:
vels.index.droplevel('m')

In [ ]:

vels.boxplot(by = 'code', figsize = (20, 10), fontsize = 12)
# vels.groupby('code').boxplot()

In [ ]:
# vels.groupby('code').plot(kind = 'scatter', grid = True, x = 'vp_0', y = 'vs_0', figsize = (10, 10))
import matplotlib.pyplot as plt

groups = vels.groupby('code')
fig, ax = plt.subplots(figsize = (10,10))
for name, group in groups:
    ax.scatter(group.vp_0, group.vs_0, marker='.', label=name, alpha = 0.25, cmap = 'viridis')
ax.grid(c = 'grey')
ax.set_xlabel('Vp', fontsize = 12, fontweight = 'bold')
ax.set_ylabel('Vs', fontsize = 12, fontweight = 'bold')
plt.legend()

In [ ]:
vels = data.loc[:, ['vp_0', 'vs_0']].copy()
idx = (vels<0).any(axis = 1)
vels.loc[idx, :].index.get_level_values('tag').unique()

In [ ]:
sum(idx)

In [ ]:
vels.shape[0] - 99